# Baseball rules engine

May 17, 2024

## Objective of this Jupyter notebook
Problem scope: I need to convert baseball rules into code for how Outs are obtained in baserunning, including:
- Force plays
- Tag plays
- Tagging up on FB
- Scoring a run

I'll ignore tertiary rules for now (e.g., time plays).

My instinct is that force plays and tag plays are States... but a force can be removed, and it can apply to R1 but not R3... so maybe not.

### Outside the scope of this notebook
- Advancing runners -- but maybe I can remove a force and get it to tell me what happens.
- Real time / real defense -- I can set up an initial state, then a notional 'ball in play' state. But I'll assume there is only 1 fielder with the ball and he can tag any runner and any base. Time doesn't pass, just situational increments.
- Number of Outs -- 2 outs vs 1 out is not in-scope 

## 1. Create baserunner object

In [120]:
last_obtained = 6
next_obtainable = 1
preceding_obtainable = 0

statuses = [last_obtained, next_obtainable, preceding_obtainable]

statuses[0]

6

In [205]:
class Baserunner:
    
    def __init__(self, name, curr_base = 0):
        
        self.name = name
                
        self.master_status = {'last_obtained': curr_base, 'next_obtainable': None, 'preceding_obtainable': None}        
        self.bases_centric_status = {1: None, 2: None, 3: None, 4: None}
        
        self.out_status = False
        self.force_status = False
        
        
    def get_name(self):
        return self.name
    
    def update_base_attained(self, new_base_attained):
        self.master_status['last_obtained'] = new_base_attained
    
    
    ### Force-play status
    
    def get_force_status(self):
        return self.force_status
        
    def update_force_status(self, force_status):
        self.force_status = force_status
    
    
    ### Apply master status to what each base is to this runner 
    
    def cascade_bases_status_noForce(self):
        for base_b, _ in self.bases_centric_status.items():
            for status_r, base_r in self.master_status.items():
                
                if base_b == base_r:
                    self.bases_centric_status[base_b] = status_r
    
    def cascade_bases_status_Force(self):
        self.master_status['last_obtained'] = None
        self.master_status['preceding_obtainable'] = None
        
        
    def update_master_status(self):
        curr_base = self.master_status['last_obtained']
        
        self.master_status['next_obtainable'] = curr_base + 1
        self.master_status['preceding_obtainable'] = curr_base -1
        
                    
    ### Tracer / print the meat of this
    def print_bases_status(self):

        #self.cascade_bases_status_noForce()
        
        if self.force_status:
            self.cascade_bases_status_Force()
            
        self.cascade_bases_status_noForce()
                   
        print(f"{self.name}: ", end = "")

        for base_b, status_b in self.bases_centric_status.items():
            print(f"{base_b}B: {status_b} ", end = " | ")
                  

## 1.1 Helper func to instantiate 4 baserunners

In [111]:
def make_runners():
    names = ['Isaac', 'Jack', 'Josh', 'Romo']
    
    return [Baserunner(name) for name in names]

### 1.2 Helper func to show who's on what base

In [190]:
def whosOnFirst(bases):
            
    for base_id, base_object in bases.items():
        
        if base_object:
            name = base_object.get_name()
            print(f"{base_id}B: {name}", end = "  ")
            
        else:
            print(f"{base_id}B: -", end = "  ")
        
        print()

## 2. Manually place runners on base

In [200]:
def manually_place_runners(bases):
    
    runners = make_runners()
    
    for base in range(3, 0, -1):

        if bases[base]:
            bases[base] = runners.pop(0)
            #bases[base].

    return bases

#### Test it

In [210]:
bases = {1: True, 2: False, 3: True, 4: False}

bases = manually_place_runners(bases)

whosOnFirst(bases)

1B: Jack  
2B: -  
3B: Isaac  
4B: -  


## 2.1. Place runners on base even more manually
- Manually set the last, next and preceding bases 

In [220]:
isaac = Baserunner('Isaac')
jack = Baserunner('Jack')

isaac.update_base_attained(3)
jack.update_base_attained(1)

#isaac.update_force_status(True)
jack.update_force_status(True)

isaac.update_master_status()
jack.update_master_status()

bases = {1: jack, 2: False, 3: isaac, 4: False}


if 1 == 1:

    print()
    whosOnFirst(bases)
    print("\n************************** \n")

    for base, runner in bases.items():

        if runner:
            
            runner.print_bases_status()
            print()


1B: Jack  
2B: -  
3B: Isaac  
4B: -  

************************** 

Jack: 1B: None  | 2B: next_obtainable  | 3B: None  | 4B: None  | 
Isaac: 1B: None  | 2B: preceding_obtainable  | 3B: last_obtained  | 4B: next_obtainable  | 


## 3. Identify forces
Just need to check 1B and 2B:
- If a runner is on 1B, then if there is an R2 he would be forced
- 